# Import Library

In [1]:
import numpy as np
import pandas as pd

import s2cell
from datetime import datetime as dt
from datetime import timedelta
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from math import sin, cos
import itertools

In [2]:
pd.options.display.max_columns = 500

# Load Dataset

In [3]:
alerts = pd.read_csv('alers_clean2.csv')

In [4]:
alerts['datehour'] = alerts['date'] + ' ' + alerts['hour'].astype(str)
alerts['datehour'] = alerts['datehour'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H'))

In [5]:
alerts['datehour_1hour'] = alerts['datehour'] - timedelta(hours=1)
alerts['datehour_3hour'] = alerts['datehour'] - timedelta(hours=3)
alerts['datehour_1day'] = alerts['datehour'] - timedelta(days=1)
alerts['datehour_3day'] = alerts['datehour'] - timedelta(days=3)
alerts['datehour_7day'] = alerts['datehour'] - timedelta(days=7)

In [6]:
alerts

,s2parent,s2token_15,date,hour,Type,SubType,N_Alerts,Road_Type,Reliability,datehour,datehour_1hour,datehour_3hour,datehour_1day,datehour_3day,datehour_7day
0,2e68c263,2e68c2624,2020-09-29,17,{'JAM'},{'JAM_STAND_STILL_TRAFFIC'},32,{2},5.906250,2020-09-29 17:00:00,2020-09-29 16:00:00,2020-09-29 14:00:00,2020-09-28 17:00:00,2020-09-26 17:00:00,2020-09-22 17:00:00
1,2e68c263,2e68c2624,2020-10-08,16,{'JAM'},{'JAM_HEAVY_TRAFFIC'},18,{2},5.000000,2020-10-08 16:00:00,2020-10-08 15:00:00,2020-10-08 13:00:00,2020-10-07 16:00:00,2020-10-05 16:00:00,2020-10-01 16:00:00
2,2e68c263,2e68c2624,2020-10-20,16,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000,2020-10-20 16:00:00,2020-10-20 15:00:00,2020-10-20 13:00:00,2020-10-19 16:00:00,2020-10-17 16:00:00,2020-10-13 16:00:00
3,2e68c263,2e68c2624,2020-11-09,17,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000,2020-11-09 17:00:00,2020-11-09 16:00:00,2020-11-09 14:00:00,2020-11-08 17:00:00,2020-11-06 17:00:00,2020-11-02 17:00:00
4,2e68c263,2e68c2624,2020-11-13,17,{'JAM'},{'JAM_HEAVY_TRAFFIC'},42,{2},5.380952,2020-11-13 17:00:00,2020-11-13 16:00:00,2020-11-13 14:00:00,2020-11-12 17:00:00,2020-11-10 17:00:00,2020-11-06 17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-15,18,{'JAM'},{'JAM'},29,{1},5.000000,2020-11-15 18:00:00,2020-11-15 17:00:00,2020-11-15 15:00:00,2020-11-14 18:00:00,2020-11-12 18:00:00,2020-11-08 18:00:00
47654,2e6a276d,2e6a276cc,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,{2},5.000000,2020-09-19 15:00:00,2020-09-19 14:00:00,2020-09-19 12:00:00,2020-09-18 15:00:00,2020-09-16 15:00:00,2020-09-12 15:00:00
47655,2e6a2773,2e6a27734,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,{2},5.000000,2020-09-19 15:00:00,2020-09-19 14:00:00,2020-09-19 12:00:00,2020-09-18 15:00:00,2020-09-16 15:00:00,2020-09-12 15:00:00
47656,2e6a2773,2e6a2773c,2020-09-19,15,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,{2},5.000000,2020-09-19 15:00:00,2020-09-19 14:00:00,2020-09-19 12:00:00,2020-09-18 15:00:00,2020-09-16 15:00:00,2020-09-12 15:00:00


In [7]:
irreg = pd.read_csv('../danthon2021/irregularities.csv')
irreg = irreg.drop_duplicates()
irreg

,id,detection_date_millis,update_date_millis,street,city,is_highway,line,s2id_center,s2token_center,speed,regular_speed,delay_seconds,seconds,length,trend,type,severity,jam_level,drivers_count,alerts_count,n_thumbs_up
0,12868069,1604733149024,1604735467276,Jatiwaringin Raya,Bekasi,t,"{""line"": [{""x"": 106.91014, ""y"": -6.258107}, {""...",3344471185277583360,2e69f2d2c,13.03,17.15,299,432,1566,0,Small,5,3,13,0,0
1,12420463,1599906813144,1599909295834,Putri Tunggal,Depok,f,"{""line"": [{""x"": 106.887821, ""y"": -6.377016}, {...",3344462996922433536,2e69eb604,6.56,18.00,399,539,984,0,Small,5,3,5,0,0
2,12497533,1601728355356,1601734996933,Ir Haji Juanda,Bandung,f,"{""line"": [{""x"": 107.618629, ""y"": -6.87556}, {""...",3344176694402482176,2e68e6fc4,3.36,19.65,1185,1294,1212,1,Large,5,4,21,2,0
3,12536831,1602312860279,1602315706305,KH Muchtar Tabrani,Bekasi,f,"{""line"": [{""x"": 107.002934, ""y"": -6.216088}, {...",3344358143885836288,2e698c034,4.36,8.04,467,543,659,-1,Small,5,4,3,0,0
4,12327151,1598956623240,1598957378934,N1 Pangeran Diponegoro,Tambun Selatan,t,"{""line"": [{""x"": 107.035652, ""y"": -6.255471}, {...",3344360723013697536,2e698e5bc,4.74,16.71,423,474,625,0,Small,5,4,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352597,12778632,1604118578540,1604122323480,Exit Baranangsiang,Bogor,f,"{""line"": [{""x"": 106.815038, ""y"": -6.599074}, {...",3344421735171620864,2e69c5d94,6.45,35.82,285,318,572,1,Small,5,4,19,0,0
352601,12630879,1603511399231,1603517083284,Dr Djundjunan,Bandung,t,"{""line"": [{""x"": 107.596014, ""y"": -6.900302}, {...",3344176048009904128,2e68e665c,5.23,11.03,1155,1280,1862,1,Small,5,4,22,0,0
352602,12533036,1602304028381,1602305330602,Kranggan Raya,Bekasi,f,"{""line"": [{""x"": 106.920568, ""y"": -6.354546}, {...",3344366143262425088,2e699349c,3.17,5.74,1030,1144,1008,-1,Small,5,4,4,0,0
352603,12834829,1604377880093,1604378652932,Muhtar Raya,Depok,t,"{""line"": [{""x"": 106.772806, ""y"": -6.394618}, {...",3344460525168754688,2e69e920c,5.01,22.39,589,675,941,1,Medium,5,4,3,0,0


In [8]:
data_train = pd.read_csv('../danthon2021/data_train.csv')
data_train

,Ids,Labels
0,2e69e9384_2020-10-06_13,True
1,2e6992c7c_2020-10-02_17,True
2,2e69ef474_2020-09-13_19,True
3,2e69c5fd4_2020-10-10_15,True
4,2e6992134_2020-09-12_11,True
...,...,...
71331,2e69eea5c_2020-11-09_10,False
71332,2e69c5944_2020-10-27_12,True
71333,2e69f2cd4_2020-11-07_14,True
71334,2e68e64e4_2020-09-23_9,False


# Experiment

In [9]:
irreg['detection_date_millis'] = irreg['detection_date_millis'].apply(lambda x: dt.fromtimestamp(x/1000))
irreg['update_date_millis'] = irreg['update_date_millis'].apply(lambda x: dt.fromtimestamp(x/1000))
irreg

,id,detection_date_millis,update_date_millis,street,city,is_highway,line,s2id_center,s2token_center,speed,regular_speed,delay_seconds,seconds,length,trend,type,severity,jam_level,drivers_count,alerts_count,n_thumbs_up
0,12868069,2020-11-07 14:12:29.024,2020-11-07 14:51:07.276,Jatiwaringin Raya,Bekasi,t,"{""line"": [{""x"": 106.91014, ""y"": -6.258107}, {""...",3344471185277583360,2e69f2d2c,13.03,17.15,299,432,1566,0,Small,5,3,13,0,0
1,12420463,2020-09-12 17:33:33.144,2020-09-12 18:14:55.834,Putri Tunggal,Depok,f,"{""line"": [{""x"": 106.887821, ""y"": -6.377016}, {...",3344462996922433536,2e69eb604,6.56,18.00,399,539,984,0,Small,5,3,5,0,0
2,12497533,2020-10-03 19:32:35.356,2020-10-03 21:23:16.933,Ir Haji Juanda,Bandung,f,"{""line"": [{""x"": 107.618629, ""y"": -6.87556}, {""...",3344176694402482176,2e68e6fc4,3.36,19.65,1185,1294,1212,1,Large,5,4,21,2,0
3,12536831,2020-10-10 13:54:20.279,2020-10-10 14:41:46.305,KH Muchtar Tabrani,Bekasi,f,"{""line"": [{""x"": 107.002934, ""y"": -6.216088}, {...",3344358143885836288,2e698c034,4.36,8.04,467,543,659,-1,Small,5,4,3,0,0
4,12327151,2020-09-01 17:37:03.240,2020-09-01 17:49:38.934,N1 Pangeran Diponegoro,Tambun Selatan,t,"{""line"": [{""x"": 107.035652, ""y"": -6.255471}, {...",3344360723013697536,2e698e5bc,4.74,16.71,423,474,625,0,Small,5,4,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352597,12778632,2020-10-31 11:29:38.540,2020-10-31 12:32:03.480,Exit Baranangsiang,Bogor,f,"{""line"": [{""x"": 106.815038, ""y"": -6.599074}, {...",3344421735171620864,2e69c5d94,6.45,35.82,285,318,572,1,Small,5,4,19,0,0
352601,12630879,2020-10-24 10:49:59.231,2020-10-24 12:24:43.284,Dr Djundjunan,Bandung,t,"{""line"": [{""x"": 107.596014, ""y"": -6.900302}, {...",3344176048009904128,2e68e665c,5.23,11.03,1155,1280,1862,1,Small,5,4,22,0,0
352602,12533036,2020-10-10 11:27:08.381,2020-10-10 11:48:50.602,Kranggan Raya,Bekasi,f,"{""line"": [{""x"": 106.920568, ""y"": -6.354546}, {...",3344366143262425088,2e699349c,3.17,5.74,1030,1144,1008,-1,Small,5,4,4,0,0
352603,12834829,2020-11-03 11:31:20.093,2020-11-03 11:44:12.932,Muhtar Raya,Depok,t,"{""line"": [{""x"": 106.772806, ""y"": -6.394618}, {...",3344460525168754688,2e69e920c,5.01,22.39,589,675,941,1,Medium,5,4,3,0,0


In [10]:
irreg['detection_date_millis'] = irreg['detection_date_millis'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
irreg['update_date_millis'] = irreg['update_date_millis'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))

In [11]:
irreg['datetime_between'] = irreg.apply(lambda x: [x.detection_date_millis + timedelta(hours=i) for i in range((x.update_date_millis-x.detection_date_millis).seconds//3600+1)], axis=1)
irreg

,id,detection_date_millis,update_date_millis,street,city,is_highway,line,s2id_center,s2token_center,speed,regular_speed,delay_seconds,seconds,length,trend,type,severity,jam_level,drivers_count,alerts_count,n_thumbs_up,datetime_between
0,12868069,2020-11-07 14:00:00,2020-11-07 14:00:00,Jatiwaringin Raya,Bekasi,t,"{""line"": [{""x"": 106.91014, ""y"": -6.258107}, {""...",3344471185277583360,2e69f2d2c,13.03,17.15,299,432,1566,0,Small,5,3,13,0,0,[2020-11-07 14:00:00]
1,12420463,2020-09-12 17:00:00,2020-09-12 18:00:00,Putri Tunggal,Depok,f,"{""line"": [{""x"": 106.887821, ""y"": -6.377016}, {...",3344462996922433536,2e69eb604,6.56,18.00,399,539,984,0,Small,5,3,5,0,0,"[2020-09-12 17:00:00, 2020-09-12 18:00:00]"
2,12497533,2020-10-03 19:00:00,2020-10-03 21:00:00,Ir Haji Juanda,Bandung,f,"{""line"": [{""x"": 107.618629, ""y"": -6.87556}, {""...",3344176694402482176,2e68e6fc4,3.36,19.65,1185,1294,1212,1,Large,5,4,21,2,0,"[2020-10-03 19:00:00, 2020-10-03 20:00:00, 202..."
3,12536831,2020-10-10 13:00:00,2020-10-10 14:00:00,KH Muchtar Tabrani,Bekasi,f,"{""line"": [{""x"": 107.002934, ""y"": -6.216088}, {...",3344358143885836288,2e698c034,4.36,8.04,467,543,659,-1,Small,5,4,3,0,0,"[2020-10-10 13:00:00, 2020-10-10 14:00:00]"
4,12327151,2020-09-01 17:00:00,2020-09-01 17:00:00,N1 Pangeran Diponegoro,Tambun Selatan,t,"{""line"": [{""x"": 107.035652, ""y"": -6.255471}, {...",3344360723013697536,2e698e5bc,4.74,16.71,423,474,625,0,Small,5,4,11,0,0,[2020-09-01 17:00:00]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352597,12778632,2020-10-31 11:00:00,2020-10-31 12:00:00,Exit Baranangsiang,Bogor,f,"{""line"": [{""x"": 106.815038, ""y"": -6.599074}, {...",3344421735171620864,2e69c5d94,6.45,35.82,285,318,572,1,Small,5,4,19,0,0,"[2020-10-31 11:00:00, 2020-10-31 12:00:00]"
352601,12630879,2020-10-24 10:00:00,2020-10-24 12:00:00,Dr Djundjunan,Bandung,t,"{""line"": [{""x"": 107.596014, ""y"": -6.900302}, {...",3344176048009904128,2e68e665c,5.23,11.03,1155,1280,1862,1,Small,5,4,22,0,0,"[2020-10-24 10:00:00, 2020-10-24 11:00:00, 202..."
352602,12533036,2020-10-10 11:00:00,2020-10-10 11:00:00,Kranggan Raya,Bekasi,f,"{""line"": [{""x"": 106.920568, ""y"": -6.354546}, {...",3344366143262425088,2e699349c,3.17,5.74,1030,1144,1008,-1,Small,5,4,4,0,0,[2020-10-10 11:00:00]
352603,12834829,2020-11-03 11:00:00,2020-11-03 11:00:00,Muhtar Raya,Depok,t,"{""line"": [{""x"": 106.772806, ""y"": -6.394618}, {...",3344460525168754688,2e69e920c,5.01,22.39,589,675,941,1,Medium,5,4,3,0,0,[2020-11-03 11:00:00]


In [12]:
irreg_clean = irreg.explode('datetime_between')
irreg_clean = irreg_clean.rename({'s2token_center':'s2token_15','datetime_between':'datehour'}, axis=1)
irreg_clean['s2parent'] = irreg_clean['s2token_15'].apply(lambda x: s2cell.token_to_parent_token(x))
irreg_clean

,id,detection_date_millis,update_date_millis,street,city,is_highway,line,s2id_center,s2token_15,speed,regular_speed,delay_seconds,seconds,length,trend,type,severity,jam_level,drivers_count,alerts_count,n_thumbs_up,datehour,s2parent
0,12868069,2020-11-07 14:00:00,2020-11-07 14:00:00,Jatiwaringin Raya,Bekasi,t,"{""line"": [{""x"": 106.91014, ""y"": -6.258107}, {""...",3344471185277583360,2e69f2d2c,13.03,17.15,299,432,1566,0,Small,5,3,13,0,0,2020-11-07 14:00:00,2e69f2d3
1,12420463,2020-09-12 17:00:00,2020-09-12 18:00:00,Putri Tunggal,Depok,f,"{""line"": [{""x"": 106.887821, ""y"": -6.377016}, {...",3344462996922433536,2e69eb604,6.56,18.00,399,539,984,0,Small,5,3,5,0,0,2020-09-12 17:00:00,2e69eb61
1,12420463,2020-09-12 17:00:00,2020-09-12 18:00:00,Putri Tunggal,Depok,f,"{""line"": [{""x"": 106.887821, ""y"": -6.377016}, {...",3344462996922433536,2e69eb604,6.56,18.00,399,539,984,0,Small,5,3,5,0,0,2020-09-12 18:00:00,2e69eb61
2,12497533,2020-10-03 19:00:00,2020-10-03 21:00:00,Ir Haji Juanda,Bandung,f,"{""line"": [{""x"": 107.618629, ""y"": -6.87556}, {""...",3344176694402482176,2e68e6fc4,3.36,19.65,1185,1294,1212,1,Large,5,4,21,2,0,2020-10-03 19:00:00,2e68e6fd
2,12497533,2020-10-03 19:00:00,2020-10-03 21:00:00,Ir Haji Juanda,Bandung,f,"{""line"": [{""x"": 107.618629, ""y"": -6.87556}, {""...",3344176694402482176,2e68e6fc4,3.36,19.65,1185,1294,1212,1,Large,5,4,21,2,0,2020-10-03 20:00:00,2e68e6fd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352601,12630879,2020-10-24 10:00:00,2020-10-24 12:00:00,Dr Djundjunan,Bandung,t,"{""line"": [{""x"": 107.596014, ""y"": -6.900302}, {...",3344176048009904128,2e68e665c,5.23,11.03,1155,1280,1862,1,Small,5,4,22,0,0,2020-10-24 12:00:00,2e68e665
352602,12533036,2020-10-10 11:00:00,2020-10-10 11:00:00,Kranggan Raya,Bekasi,f,"{""line"": [{""x"": 106.920568, ""y"": -6.354546}, {...",3344366143262425088,2e699349c,3.17,5.74,1030,1144,1008,-1,Small,5,4,4,0,0,2020-10-10 11:00:00,2e699349
352603,12834829,2020-11-03 11:00:00,2020-11-03 11:00:00,Muhtar Raya,Depok,t,"{""line"": [{""x"": 106.772806, ""y"": -6.394618}, {...",3344460525168754688,2e69e920c,5.01,22.39,589,675,941,1,Medium,5,4,3,0,0,2020-11-03 11:00:00,2e69e921
352608,12912757,2020-11-10 17:00:00,2020-11-10 18:00:00,Haji Abu Bakar,Tambun Selatan,f,"{""line"": [{""x"": 107.055937, ""y"": -6.265914}, {...",3344362200482447360,2e698fb3c,3.28,14.03,582,666,609,-1,Medium,5,4,3,0,0,2020-11-10 17:00:00,2e698fb3


In [13]:
grouped = (irreg_clean.groupby(['s2parent', 's2token_15', 'datehour'])
           .agg(Alerts_Count=('alerts_count', 'sum'), 
                Drivers_Count=('drivers_count', 'sum'), 
                Jam_Level=('jam_level', 'mean'), 
                Is_Highway=('is_highway', set) ))
grouped

Alerts_Count  Drivers_Count  \
s2parent s2token_15 datehour                                           
2e68c259 2e68c2584  2020-11-08 09:00:00             0              4   
         2e68c258c  2020-11-08 09:00:00             0              2   
2e68c25f 2e68c25f4  2020-11-08 09:00:00             0             10   
         2e68c25fc  2020-11-08 09:00:00             0             20   
2e68c261 2e68c261c  2020-11-08 09:00:00             0              7   
...                                               ...            ...   
2e6a20ef 2e6a20ee4  2020-09-28 09:00:00             0             16   
                    2020-11-05 15:00:00             0             40   
         2e6a20ef4  2020-09-28 08:00:00             0              4   
                    2020-11-21 10:00:00             0             12   
                    2020-11-21 16:00:00             0             26   

                                         Jam_Level Is_Highway  
s2parent s2token_15 datehour                                   
2e68c259 2e68c2584  2020-11-08 09:00:00   1.000000        {f}  
         2e68c258c  2020-11-08 09:00:00   1.000000        {f}  
2e68c25f 2e68c25f4  2020-11-08 09:00:00   1.000000        {f}  
         2e68c25fc  2020-11-08 09:00:00   1.000000        {f}  
2e68c261 2e68c261c  2020-11-08 09:00:00   1.000000        {f}  
...                                            ...        ...  
2e6a20ef 2e6a20ee4  2020-09-28 09:00:00   3.000000        {f}  
                    2020-11-05 15:00:00   4.000000        {f}  
         2e6a20ef4  2020-09-28 08:00:00   4.000000        {f}  
                    2020-11-21 10:00:00   3.000000        {f}  
                    2020-11-21 16:00:00   3.615385        {f}  

[34390 rows x 4 columns]

In [14]:
grouped['Is_Highway'] = grouped['Is_Highway'].apply(lambda x: list(x)[0])
grouped

Alerts_Count  Drivers_Count  \
s2parent s2token_15 datehour                                           
2e68c259 2e68c2584  2020-11-08 09:00:00             0              4   
         2e68c258c  2020-11-08 09:00:00             0              2   
2e68c25f 2e68c25f4  2020-11-08 09:00:00             0             10   
         2e68c25fc  2020-11-08 09:00:00             0             20   
2e68c261 2e68c261c  2020-11-08 09:00:00             0              7   
...                                               ...            ...   
2e6a20ef 2e6a20ee4  2020-09-28 09:00:00             0             16   
                    2020-11-05 15:00:00             0             40   
         2e6a20ef4  2020-09-28 08:00:00             0              4   
                    2020-11-21 10:00:00             0             12   
                    2020-11-21 16:00:00             0             26   

                                         Jam_Level Is_Highway  
s2parent s2token_15 datehour                                   
2e68c259 2e68c2584  2020-11-08 09:00:00   1.000000          f  
         2e68c258c  2020-11-08 09:00:00   1.000000          f  
2e68c25f 2e68c25f4  2020-11-08 09:00:00   1.000000          f  
         2e68c25fc  2020-11-08 09:00:00   1.000000          f  
2e68c261 2e68c261c  2020-11-08 09:00:00   1.000000          f  
...                                            ...        ...  
2e6a20ef 2e6a20ee4  2020-09-28 09:00:00   3.000000          f  
                    2020-11-05 15:00:00   4.000000          f  
         2e6a20ef4  2020-09-28 08:00:00   4.000000          f  
                    2020-11-21 10:00:00   3.000000          f  
                    2020-11-21 16:00:00   3.615385          f  

[34390 rows x 4 columns]

In [15]:
irreg_grouped = grouped.reset_index()
irreg_grouped

,s2parent,s2token_15,datehour,Alerts_Count,Drivers_Count,Jam_Level,Is_Highway
0,2e68c259,2e68c2584,2020-11-08 09:00:00,0,4,1.000000,f
1,2e68c259,2e68c258c,2020-11-08 09:00:00,0,2,1.000000,f
2,2e68c25f,2e68c25f4,2020-11-08 09:00:00,0,10,1.000000,f
3,2e68c25f,2e68c25fc,2020-11-08 09:00:00,0,20,1.000000,f
4,2e68c261,2e68c261c,2020-11-08 09:00:00,0,7,1.000000,f
...,...,...,...,...,...,...,...
34385,2e6a20ef,2e6a20ee4,2020-09-28 09:00:00,0,16,3.000000,f
34386,2e6a20ef,2e6a20ee4,2020-11-05 15:00:00,0,40,4.000000,f
34387,2e6a20ef,2e6a20ef4,2020-09-28 08:00:00,0,4,4.000000,f
34388,2e6a20ef,2e6a20ef4,2020-11-21 10:00:00,0,12,3.000000,f


In [16]:
alerts_main = alerts[['s2parent', 's2token_15', 'datehour', 'Type', 'SubType', 'N_Alerts', 'Reliability', 'Road_Type']]
alerts_main

,s2parent,s2token_15,datehour,Type,SubType,N_Alerts,Reliability,Road_Type
0,2e68c263,2e68c2624,2020-09-29 17:00:00,{'JAM'},{'JAM_STAND_STILL_TRAFFIC'},32,5.906250,{2}
1,2e68c263,2e68c2624,2020-10-08 16:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},18,5.000000,{2}
2,2e68c263,2e68c2624,2020-10-20 16:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,5.000000,{2}
3,2e68c263,2e68c2624,2020-11-09 17:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,5.000000,{2}
4,2e68c263,2e68c2624,2020-11-13 17:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},42,5.380952,{2}
...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-15 18:00:00,{'JAM'},{'JAM'},29,5.000000,{1}
47654,2e6a276d,2e6a276cc,2020-09-19 15:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,5.000000,{2}
47655,2e6a2773,2e6a27734,2020-09-19 15:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,5.000000,{2}
47656,2e6a2773,2e6a2773c,2020-09-19 15:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,5.000000,{2}


In [17]:
irreg_alerts = pd.merge(alerts_main, irreg_grouped, on=['s2parent', 's2token_15', 'datehour'], how='left')
irreg_alerts

,s2parent,s2token_15,datehour,Type,SubType,N_Alerts,Reliability,Road_Type,Alerts_Count,Drivers_Count,Jam_Level,Is_Highway
0,2e68c263,2e68c2624,2020-09-29 17:00:00,{'JAM'},{'JAM_STAND_STILL_TRAFFIC'},32,5.906250,{2},NaN,NaN,NaN,NaN
1,2e68c263,2e68c2624,2020-10-08 16:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},18,5.000000,{2},NaN,NaN,NaN,NaN
2,2e68c263,2e68c2624,2020-10-20 16:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,5.000000,{2},NaN,NaN,NaN,NaN
3,2e68c263,2e68c2624,2020-11-09 17:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,5.000000,{2},NaN,NaN,NaN,NaN
4,2e68c263,2e68c2624,2020-11-13 17:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},42,5.380952,{2},NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-15 18:00:00,{'JAM'},{'JAM'},29,5.000000,{1},NaN,NaN,NaN,NaN
47654,2e6a276d,2e6a276cc,2020-09-19 15:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,5.000000,{2},NaN,NaN,NaN,NaN
47655,2e6a2773,2e6a27734,2020-09-19 15:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},30,5.000000,{2},NaN,NaN,NaN,NaN
47656,2e6a2773,2e6a2773c,2020-09-19 15:00:00,{'JAM'},{'JAM_HEAVY_TRAFFIC'},31,5.000000,{2},NaN,NaN,NaN,NaN


In [18]:
irreg_alerts_Type = pd.DataFrame(irreg_alerts.Type.apply(eval).to_list(), columns=[f'Type{x}' for x in range(3)])
irreg_alerts_SubType = pd.DataFrame(irreg_alerts.SubType.apply(eval).to_list(), columns=[f'SubType{x}' for x in range(6)])
irreg_alerts_Road_Type = pd.DataFrame(irreg_alerts.Road_Type.apply(eval).to_list(), columns=[f'Road_Type{x}' for x in range(3)])

irreg_alerts_Road_Type

,Road_Type0,Road_Type1,Road_Type2
0,2,NaN,NaN
1,2,NaN,NaN
2,2,NaN,NaN
3,2,NaN,NaN
4,2,NaN,NaN
...,...,...,...
47653,1,NaN,NaN
47654,2,NaN,NaN
47655,2,NaN,NaN
47656,2,NaN,NaN


In [44]:
type_sub_road = [f'Type{x}' for x in range(3)] + [f'SubType{x}' for x in range(6)]
type_sub_road

['Type0',
 'Type1',
 'Type2',
 'SubType0',
 'SubType1',
 'SubType2',
 'SubType3',
 'SubType4',
 'SubType5']

In [45]:
highway_roadtype = {1:'t', 2:'f', 4:'t', 5:'f', 6:'t', 7:'f', 8:'f', 17:'f', 20:'f', 21:'f', 22:'t'}

In [54]:
irreg_alerts_clean = pd.concat([irreg_alerts, irreg_alerts_Type, irreg_alerts_SubType, irreg_alerts_Road_Type], axis=1)
irreg_alerts_clean = irreg_alerts_clean.drop(['Type', 'SubType', 'Road_Type'], axis=1)
irreg_alerts_clean[['Alerts_Count', 'Drivers_Count', 'Jam_Level']] = irreg_alerts_clean[['Alerts_Count', 'Drivers_Count', 'Jam_Level']].fillna(0)
irreg_alerts_clean[type_sub_road] = irreg_alerts_clean[type_sub_road].fillna('NS')
irreg_alerts_clean[[f'Road_Type{x}' for x in range(3)]] = irreg_alerts_clean[[f'Road_Type{x}' for x in range(3)]].fillna(0)
irreg_alerts_clean.loc[irreg_alerts_clean.Is_Highway.isnull(), 'Is_Highway'] = irreg_alerts_clean.Road_Type0.map(highway_roadtype)
irreg_alerts_clean

,s2parent,s2token_15,datehour,N_Alerts,Reliability,Alerts_Count,Drivers_Count,Jam_Level,Is_Highway,Type0,Type1,Type2,SubType0,SubType1,SubType2,SubType3,SubType4,SubType5,Road_Type0,Road_Type1,Road_Type2
0,2e68c263,2e68c2624,2020-09-29 17:00:00,32,5.906250,0.0,0.0,0.0,f,JAM,NS,NS,JAM_STAND_STILL_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0
1,2e68c263,2e68c2624,2020-10-08 16:00:00,18,5.000000,0.0,0.0,0.0,f,JAM,NS,NS,JAM_HEAVY_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0
2,2e68c263,2e68c2624,2020-10-20 16:00:00,31,5.000000,0.0,0.0,0.0,f,JAM,NS,NS,JAM_HEAVY_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0
3,2e68c263,2e68c2624,2020-11-09 17:00:00,31,5.000000,0.0,0.0,0.0,f,JAM,NS,NS,JAM_HEAVY_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0
4,2e68c263,2e68c2624,2020-11-13 17:00:00,42,5.380952,0.0,0.0,0.0,f,JAM,NS,NS,JAM_HEAVY_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-15 18:00:00,29,5.000000,0.0,0.0,0.0,t,JAM,NS,NS,JAM,NS,NS,NS,NS,NS,1,0.0,0.0
47654,2e6a276d,2e6a276cc,2020-09-19 15:00:00,30,5.000000,0.0,0.0,0.0,f,JAM,NS,NS,JAM_HEAVY_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0
47655,2e6a2773,2e6a27734,2020-09-19 15:00:00,30,5.000000,0.0,0.0,0.0,f,JAM,NS,NS,JAM_HEAVY_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0
47656,2e6a2773,2e6a2773c,2020-09-19 15:00:00,31,5.000000,0.0,0.0,0.0,f,JAM,NS,NS,JAM_HEAVY_TRAFFIC,NS,NS,NS,NS,NS,2,0.0,0.0


In [57]:
parent_road_type = irreg_alerts_clean.groupby('s2parent').agg(Road_Type_Parent=('Road_Type0', list)).reset_index()
parent_road_type['roadtype'] = parent_road_type.Road_Type_Parent.apply(lambda x: max(set(x), key=x.count))

parent_road_type

,s2parent,Road_Type_Parent,roadtype
0,2e68c263,"[2, 2, 2, 2, 2, 4]",2
1,2e68c281,"[2, 2]",2
2,2e68c283,[2],2
3,2e68c285,[2],2
4,2e68c287,[1],1
...,...,...,...
2138,2e6a2739,[1],1
2139,2e6a2741,[1],1
2140,2e6a276d,[2],2
2141,2e6a2773,"[2, 2]",2


In [63]:
mapping_parent_road = {row.s2parent:row.roadtype for idx, row in parent_road_type.iterrows()}
mapping_parent_road

{'2e68c263': 2,
 '2e68c281': 2,
 '2e68c283': 2,
 '2e68c285': 2,
 '2e68c287': 1,
 '2e68c28b': 2,
 '2e68c28f': 1,
 '2e68c291': 1,
 '2e68c295': 2,
 '2e68c29b': 1,
 '2e68c29d': 1,
 '2e68c29f': 1,
 '2e68c2a5': 2,
 '2e68c2a7': 6,
 '2e68c2a9': 1,
 '2e68c2ad': 2,
 '2e68c2af': 17,
 '2e68c2b1': 1,
 '2e68c2b3': 1,
 '2e68c2b5': 2,
 '2e68c2b9': 6,
 '2e68c2bb': 6,
 '2e68c2bd': 1,
 '2e68c2bf': 2,
 '2e68c2c5': 6,
 '2e68c2c7': 6,
 '2e68c2c9': 2,
 '2e68c2cb': 2,
 '2e68c2cd': 1,
 '2e68c2d1': 2,
 '2e68c2d5': 1,
 '2e68c2d7': 6,
 '2e68c2d9': 6,
 '2e68c2db': 6,
 '2e68c2dd': 1,
 '2e68c2e5': 1,
 '2e68c2e7': 1,
 '2e68c2f5': 1,
 '2e68c2fb': 1,
 '2e68c327': 6,
 '2e68c329': 6,
 '2e68c32b': 1,
 '2e68c32d': 1,
 '2e68dcd1': 1,
 '2e68dcd5': 1,
 '2e68dd1f': 1,
 '2e68dd23': 1,
 '2e68dd25': 1,
 '2e68dd29': 1,
 '2e68dd2d': 2,
 '2e68dd2f': 6,
 '2e68dd31': 2,
 '2e68dd33': 1,
 '2e68dd35': 2,
 '2e68dd37': 2,
 '2e68dd39': 6,
 '2e68dd3b': 6,
 '2e68dd41': 6,
 '2e68dd43': 6,
 '2e68dd47': 6,
 '2e68dd49': 1,
 '2e68dd4d': 2,
 '2e68d

In [58]:
irreg_alerts_clean = irreg_alerts_clean.drop(['s2parent'], axis=1)

In [56]:
irreg_alerts_clean.to_csv('irreg_alerts_clean.csv', index=0)

In [64]:
train = data_train.copy()
train[['s2token_15', 'date', 'hour']] = train.Ids.str.split('_', expand=True)
train['s2parent'] = train.s2token_15.apply(lambda x: s2cell.token_to_parent_token(x))
train['datehour'] = train['date'] + '_' + train['hour']
train['datehour'] = train['datehour'].apply(lambda x: dt.strptime(x, '%Y-%m-%d_%H'))
for x in [1, 3, 12, 24, 72, 168]:
    train[f'datehour_{x}hour'] = train['datehour'] - timedelta(hours=x)
    
for x in [1, 3, 12, 24, 72, 168]:
    train = pd.merge(train, irreg_alerts_clean, how='left', left_on=['s2token_15', f'datehour_{x}hour'], 
                     right_on=['s2token_15', 'datehour'], suffixes=('', f'_{x}hour'))

fill_0 = ([x for x in train.columns if 'N_Alerts' in x ] + 
          [x for x in train.columns if 'Reliability' in x ] + 
          [x for x in train.columns if 'Alerts_Count' in x ] + 
          [x for x in train.columns if 'Drivers_Count' in x ] + 
          [x for x in train.columns if 'Jam_Level' in x ] + 
          [x for x in train.columns if 'Road_Type1' in x or 'Road_Type2' in x ])
train[fill_0]= train[fill_0].fillna(0)

fill_NS = ([x for x in train.columns if 'Type' in x and 'Road' not in x ] + 
           [x for x in train.columns if 'SubType' in x ])
train[fill_NS]= train[fill_NS].fillna('NS')

### Roadtype0 and is_highway based on mapping

datehour_cols = [x for x in train.columns if 'datehour_' in x ]
train = train.drop(datehour_cols, axis=1)

train

,Ids,Labels,s2token_15,date,hour,s2parent,datehour,N_Alerts,Reliability,Alerts_Count,Drivers_Count,Jam_Level,Is_Highway,Type0,Type1,Type2,SubType0,SubType1,SubType2,SubType3,SubType4,SubType5,Road_Type0,Road_Type1,Road_Type2,N_Alerts_3hour,Reliability_3hour,Alerts_Count_3hour,Drivers_Count_3hour,Jam_Level_3hour,Is_Highway_3hour,Type0_3hour,Type1_3hour,Type2_3hour,SubType0_3hour,SubType1_3hour,SubType2_3hour,SubType3_3hour,SubType4_3hour,SubType5_3hour,Road_Type0_3hour,Road_Type1_3hour,Road_Type2_3hour,N_Alerts_12hour,Reliability_12hour,Alerts_Count_12hour,Drivers_Count_12hour,Jam_Level_12hour,Is_Highway_12hour,Type0_12hour,Type1_12hour,Type2_12hour,SubType0_12hour,SubType1_12hour,SubType2_12hour,SubType3_12hour,SubType4_12hour,SubType5_12hour,Road_Type0_12hour,Road_Type1_12hour,Road_Type2_12hour,N_Alerts_24hour,Reliability_24hour,Alerts_Count_24hour,Drivers_Count_24hour,Jam_Level_24hour,Is_Highway_24hour,Type0_24hour,Type1_24hour,Type2_24hour,SubType0_24hour,SubType1_24hour,SubType2_24hour,SubType3_24hour,SubType4_24hour,SubType5_24hour,Road_Type0_24hour,Road_Type1_24hour,Road_Type2_24hour,N_Alerts_72hour,Reliability_72hour,Alerts_Count_72hour,Drivers_Count_72hour,Jam_Level_72hour,Is_Highway_72hour,Type0_72hour,Type1_72hour,Type2_72hour,SubType0_72hour,SubType1_72hour,SubType2_72hour,SubType3_72hour,SubType4_72hour,SubType5_72hour,Road_Type0_72hour,Road_Type1_72hour,Road_Type2_72hour,N_Alerts_168hour,Reliability_168hour,Alerts_Count_168hour,Drivers_Count_168hour,Jam_Level_168hour,Is_Highway_168hour,Type0_168hour,Type1_168hour,Type2_168hour,SubType0_168hour,SubType1_168hour,SubType2_168hour,SubType3_168hour,SubType4_168hour,SubType5_168hour,Road_Type0_168hour,Road_Type1_168hour,Road_Type2_168hour
0,2e69e9384_2020-10-06_13,True,2e69e9384,2020-10-06,13,2e69e939,2020-10-06 13:00:00,0.0,0.0,0.0,0.0,0.00,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0
1,2e6992c7c_2020-10-02_17,True,2e6992c7c,2020-10-02,17,2e6992c7,2020-10-02 17:00:00,0.0,0.0,0.0,0.0,0.00,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0
2,2e69ef474_2020-09-13_19,True,2e69ef474,2020-09-13,19,2e69ef47,2020-09-13 19:00:00,0.0,0.0,0.0,0.0,0.00,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0
3,2e69c5fd4_2020-10-10_15,True,2e69c5fd4,2020-10-10,15,2e69c5fd,2020-10-10 15:00:00,0.0,0.0,0.0,0.0,0.00,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0
4,2e6992134_2020-09-12_11,True,2e6992134,2020-09-12,11,2e699213,2020-09-12 11:00:00,0.0,0.0,0.0,0.0,0.00,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,NS,NS,NS,NS,NS,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NS,NS,NS,NS,

# Scratchpad

In [18]:
alerts_1hour = alerts[['s2parent', 's2token_15', 'datehour_1hour']]
alerts_1hour = pd.merge(alerts_1hour, alerts_main, how='left', left_on=['s2parent', 's2token_15','datehour_1hour'], 
                        right_on=['s2parent', 's2token_15','datehour'])
alerts_1hour = alerts_1hour.drop(['datehour', 's2parent','s2token_15'], axis=1)
alerts_1hour[['Type', 'SubType']] = alerts_1hour[['Type', 'SubType']].fillna('{}')
alerts_1hour[['N_Alerts', 'Reliability']] = alerts_1hour[['N_Alerts', 'Reliability']].fillna(0)
alerts_1hour

,datehour_1hour,Type,SubType,N_Alerts,Reliability,Road_Type
0,2020-09-29 16:00:00,{},{},0.0,0.0,NaN
1,2020-10-08 15:00:00,{},{},0.0,0.0,NaN
2,2020-10-20 15:00:00,{},{},0.0,0.0,NaN
3,2020-11-09 16:00:00,{},{},0.0,0.0,NaN
4,2020-11-13 16:00:00,{},{},0.0,0.0,NaN
...,...,...,...,...,...,...
47653,2020-11-15 17:00:00,{},{},0.0,0.0,NaN
47654,2020-09-19 14:00:00,{},{},0.0,0.0,NaN
47655,2020-09-19 14:00:00,{},{},0.0,0.0,NaN
47656,2020-09-19 14:00:00,{},{},0.0,0.0,NaN


In [19]:
alerts_3hour = alerts[['s2parent', 's2token_15', 'datehour_3hour']]
alerts_3hour = pd.merge(alerts_3hour, alerts_main, how='left', left_on=['s2parent', 's2token_15','datehour_3hour'], 
                        right_on=['s2parent', 's2token_15','datehour'])
alerts_3hour

,s2parent,s2token_15,datehour_3hour,datehour,Type,SubType,N_Alerts,Reliability,Road_Type
0,2e68c263,2e68c2624,2020-09-29 14:00:00,NaT,NaN,NaN,NaN,NaN,NaN
1,2e68c263,2e68c2624,2020-10-08 13:00:00,NaT,NaN,NaN,NaN,NaN,NaN
2,2e68c263,2e68c2624,2020-10-20 13:00:00,NaT,NaN,NaN,NaN,NaN,NaN
3,2e68c263,2e68c2624,2020-11-09 14:00:00,NaT,NaN,NaN,NaN,NaN,NaN
4,2e68c263,2e68c2624,2020-11-13 14:00:00,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-15 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47654,2e6a276d,2e6a276cc,2020-09-19 12:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47655,2e6a2773,2e6a27734,2020-09-19 12:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47656,2e6a2773,2e6a2773c,2020-09-19 12:00:00,NaT,NaN,NaN,NaN,NaN,NaN


In [20]:
alerts_1day = alerts[['s2parent', 's2token_15', 'datehour_1day']]
alerts_1day = pd.merge(alerts_1day, alerts_main, how='left', left_on=['s2parent', 's2token_15','datehour_1day'], 
                        right_on=['s2parent', 's2token_15','datehour'])
alerts_1day

,s2parent,s2token_15,datehour_1day,datehour,Type,SubType,N_Alerts,Reliability,Road_Type
0,2e68c263,2e68c2624,2020-09-28 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
1,2e68c263,2e68c2624,2020-10-07 16:00:00,NaT,NaN,NaN,NaN,NaN,NaN
2,2e68c263,2e68c2624,2020-10-19 16:00:00,NaT,NaN,NaN,NaN,NaN,NaN
3,2e68c263,2e68c2624,2020-11-08 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
4,2e68c263,2e68c2624,2020-11-12 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-14 18:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47654,2e6a276d,2e6a276cc,2020-09-18 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47655,2e6a2773,2e6a27734,2020-09-18 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47656,2e6a2773,2e6a2773c,2020-09-18 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN


In [21]:
alerts_3day = alerts[['s2parent', 's2token_15', 'datehour_3day']]
alerts_3day = pd.merge(alerts_3day, alerts_main, how='left', left_on=['s2parent', 's2token_15','datehour_3day'], 
                        right_on=['s2parent', 's2token_15','datehour'])
alerts_3day

,s2parent,s2token_15,datehour_3day,datehour,Type,SubType,N_Alerts,Reliability,Road_Type
0,2e68c263,2e68c2624,2020-09-26 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
1,2e68c263,2e68c2624,2020-10-05 16:00:00,NaT,NaN,NaN,NaN,NaN,NaN
2,2e68c263,2e68c2624,2020-10-17 16:00:00,NaT,NaN,NaN,NaN,NaN,NaN
3,2e68c263,2e68c2624,2020-11-06 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
4,2e68c263,2e68c2624,2020-11-10 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-12 18:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47654,2e6a276d,2e6a276cc,2020-09-16 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47655,2e6a2773,2e6a27734,2020-09-16 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47656,2e6a2773,2e6a2773c,2020-09-16 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN


In [22]:
alerts_7day = alerts[['s2parent', 's2token_15', 'datehour_7day']]
alerts_7day = pd.merge(alerts_7day, alerts_main, how='left', left_on=['s2parent', 's2token_15','datehour_7day'], 
                        right_on=['s2parent', 's2token_15','datehour'])
alerts_7day

,s2parent,s2token_15,datehour_7day,datehour,Type,SubType,N_Alerts,Reliability,Road_Type
0,2e68c263,2e68c2624,2020-09-22 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
1,2e68c263,2e68c2624,2020-10-01 16:00:00,NaT,NaN,NaN,NaN,NaN,NaN
2,2e68c263,2e68c2624,2020-10-13 16:00:00,NaT,NaN,NaN,NaN,NaN,NaN
3,2e68c263,2e68c2624,2020-11-02 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
4,2e68c263,2e68c2624,2020-11-06 17:00:00,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
47653,2e6a2741,2e6a27404,2020-11-08 18:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47654,2e6a276d,2e6a276cc,2020-09-12 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47655,2e6a2773,2e6a27734,2020-09-12 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
47656,2e6a2773,2e6a2773c,2020-09-12 15:00:00,NaT,NaN,NaN,NaN,NaN,NaN
